In [36]:
import socket
import sys
import platform
import os
from scapy.all import sniff, wrpcap, rdpcap
import datetime
import glob
import psutil
import argparse
import time

In [37]:
system = platform.system().lower()
print(system)

windows


In [38]:
INTERFACE = "Беспроводная сеть"
PROCESS = "cs2.exe"
DURATION = 60

In [39]:
def get_process_connections(process_name):
    """Получает сетевые соединения для процесса по имени."""
    connections = []
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            if proc.info['name'].lower() == process_name.lower():
                pid = proc.info['pid']
                print(f"Найден процесс {process_name} с PID {pid}")

                # Выводим все соединения для отладки
                print(f"Все соединения процесса: {psutil.Process(pid).net_connections()}")

                # Получаем соединения процесса
                for conn in psutil.Process(pid).net_connections():
                    # Включаем TCP-соединения с активным статусом
                    if (conn.type == socket.SOCK_STREAM and
                        conn.status in (psutil.CONN_ESTABLISHED, psutil.CONN_SYN_SENT, psutil.CONN_SYN_RECV)):
                        connections.append({
                            'laddr': conn.laddr.ip if conn.laddr else '',
                            'raddr': conn.raddr.ip if conn.raddr else '',
                            'lport': conn.laddr.port if conn.laddr else 0,
                            'rport': conn.raddr.port if conn.raddr else 0,
                            'type': 'tcp'
                        })
                    # Включаем UDP-сокеты (игнорируем статус)
                    elif conn.type == socket.SOCK_DGRAM:
                        connections.append({
                            'laddr': conn.laddr.ip if conn.laddr else '',
                            'raddr': conn.raddr.ip if conn.raddr else '',
                            'lport': conn.laddr.port if conn.laddr else 0,
                            'rport': conn.raddr.port if conn.raddr else 0,
                            'type': 'udp'
                        })
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            continue

    if not connections:
        print(f"Не найдены активные соединения для процесса {process_name}.")
    else:
        print(f"Найдено {len(connections)} соединений для {process_name}: {connections}")
    return connections

In [40]:
def create_bpf_filter(process_name):
    """Создает BPF-фильтр на основе соединений процесса."""
    connections = get_process_connections(process_name)
    if not connections:
        return ""

    ip_list = set()
    port_list = set()
    proto_filters = []

    for conn in connections:
        if conn['raddr']:
            ip_list.add(conn['raddr'])
        if conn['lport']:
            port_list.add(conn['lport'])
        if conn['rport']:
            port_list.add(conn['rport'])

    if not port_list:
        print("Не найдены порты для фильтрации.")
        return ""

    # Создаем фильтр по портам и протоколам
    port_filter = " or ".join(
        f"{conn['type']} port {port}" for port in port_list for conn in connections
        if conn['lport'] == port or conn['rport'] == port
    )

    if ip_list:
        ip_filter = " or ".join(f"host {ip}" for ip in ip_list)
        return f"({port_filter}) and ({ip_filter})"
    else:
        # Если нет удаленных IP, фильтруем только по портам
        return port_filter

In [41]:
def start_capture(interface, output_file, duration, process_name):
    """Захват трафика для процесса и сохранение в PCAP-файл."""
    packets = []
    start_time = time.time()

    def packet_callback(packet):
        packets.append(packet)
        print(f"Захвачен пакет: {packet.summary()}")

    # Начальный фильтр
    bpf_filter = create_bpf_filter(process_name)
    if not bpf_filter:
        print("Не удалось создать начальный BPF-фильтр. Проверьте, запущен ли процесс.")
        sys.exit(1)

    print(f"Начальный BPF-фильтр: {bpf_filter}")
    print(f"Начало захвата на интерфейсе {interface} для процесса {process_name}...")

    try:
        # Захват с периодическим обновлением фильтра
        while time.time() - start_time < duration:
            sniff(iface=interface, prn=packet_callback, timeout=30, filter=bpf_filter)
            # Обновляем фильтр каждые 30 секунд
            new_bpf_filter = create_bpf_filter(process_name)
            if new_bpf_filter and new_bpf_filter != bpf_filter:
                print(f"Обновлен BPF-фильтр: {new_bpf_filter}")
                bpf_filter = new_bpf_filter

        print(f"Сохранение {len(packets)} пакетов в файл {output_file}...")
        wrpcap(output_file, packets)
        print(f"Захват завершен. Файл сохранен в: {output_file}")

    except Exception as e:
        print(f"Ошибка при захвате трафика: {e}")
        sys.exit(1)

In [42]:
RAW_PATH = f'raw/games'
FILENAME = f"{len(os.listdir(RAW_PATH)) + 1}_{ PROCESS.replace(".exe", "") }.pcap"
OUTPUT_FILE = f"{RAW_PATH}/{FILENAME}"

In [43]:
print(OUTPUT_FILE)

raw/games/2_cs2.pcap


In [ ]:
start_capture(INTERFACE, OUTPUT_FILE, DURATION, PROCESS)

Найден процесс cs2.exe с PID 1528
Все соединения процесса: [pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_DGRAM: 2>, laddr=addr(ip='0.0.0.0', port=64433), raddr=(), status='NONE'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_DGRAM: 2>, laddr=addr(ip='0.0.0.0', port=57453), raddr=(), status='NONE'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_DGRAM: 2>, laddr=addr(ip='0.0.0.0', port=57430), raddr=(), status='NONE'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_DGRAM: 2>, laddr=addr(ip='0.0.0.0', port=57412), raddr=(), status='NONE'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_DGRAM: 2>, laddr=addr(ip='0.0.0.0', port=57452), raddr=(), status='NONE'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_DGRAM: 2>, laddr=addr(ip='0.0.0.0', port=57454), raddr=(), status='NONE'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_